In [1]:
import torch

if torch.cuda.is_available():    
    device = torch.device("cuda")
else:
    device = torch.device("cpu")

In [3]:
import pandas as pd
import math
from sklearn import preprocessing

file = "hasoc_2020_hi_train_a.xlsx"
file_test = "hindi_test_1509.csv"

df_train = pd.read_excel(file,index_col=0)

df_train = df_train.dropna()

df_test = pd.read_csv(file_test)

task = 'task1'
task_2019 = 'task_1'

file_2019_1 = pd.read_csv("hasoc2019_hi_test_gold_2919.tsv",sep='\t')
file_2019_2 = pd.read_csv("hindi_dataset.tsv",sep="\t")

sentences_2019_1 = list(file_2019_1['text'].values)
sentences_2019_2 = list(file_2019_2['text'].values)

labels_2019_1 = list(file_2019_1[task_2019].values)
labels_2019_2 = list(file_2019_2[task_2019].values)

print(len(df_train))
print(df_train.head())

total_sentences = list(df_train['text'].values)
total_labels = list(df_train['task1'].values)

total_sentences.extend(sentences_2019_1)
total_sentences.extend(sentences_2019_2)

total_labels.extend(labels_2019_1)
total_labels.extend(labels_2019_2)

test_sentences = list(df_test['text'].values)
test_labels = list(df_test[task].values)

def clean_text(sentences):
    for index,line in enumerate(sentences):
        if "\n" in line:
            sentences[index] = line.replace("\n","")
    return sentences
        
total_sentences = clean_text(total_sentences)
test_sentences = clean_text(test_sentences)

def clean_labels(labels):
    new_list= []
    for value in labels:
        new_list.append(value.strip())
    return new_list

total_labels = clean_labels(total_labels)
test_labels = clean_labels(test_labels)

le = preprocessing.LabelEncoder()
le.fit(total_labels)
encoded_labels = le.transform(total_labels)
encoded_test_labels = le.transform(test_labels)
print(set(encoded_labels))

print(len(total_sentences),len(encoded_labels),len(test_sentences),len(encoded_test_labels))

print(df_test)

2963
                                                                  text task1  \
tweet_id                                                                       
1127755185944711170  1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो\n...   HOF   
1123578938406592513  RT @Vishesh4: @jawaharyadavbjp जवाहर यादव, अगर...   NOT   
1127750010156642304  RT @FunKeyBaat: #भगवा वस्त्र पहन कर मतदान नही ...   HOF   
1127660759553597441  Yey nina khothani labafazi benu phambili Finis...   HOF   
1123487255136489472  RT @Rajeshbhanjan2: जब भी कोई सिकुलर कोंग्रेसी...   HOF   

                    task2                  ID  
tweet_id                                       
1127755185944711170  HATE  hasoc_2020_hi_2250  
1123578938406592513  NONE   hasoc_2020_hi_381  
1127750010156642304  HATE  hasoc_2020_hi_1510  
1127660759553597441  PRFN  hasoc_2020_hi_1010  
1123487255136489472  HATE   hasoc_2020_hi_331  
{0, 1}
8946 8946 663 663
                tweet_id                                               te

In [4]:
!pip install transformers

ERROR: After October 2020 you may experience errors when installing or updating packages. This is because pip will change the way that it resolves dependency conflicts.

We recommend you use --use-feature=2020-resolver to test your packages with the new resolver before it becomes the default.

huggingface-hub 0.0.17 requires packaging>=20.9, but you'll have packaging 20.4 which is incompatible.


In [5]:
import transformers

In [6]:
!pip install transformers[sentencepiece]

  Attempting uninstall: sentencepiece
    Found existing installation: sentencepiece 0.1.96
    Uninstalling sentencepiece-0.1.96:
      Successfully uninstalled sentencepiece-0.1.96


In [7]:
from transformers import BertTokenizer


tokenizer = BertTokenizer.from_pretrained('bert-base-multilingual-cased')

max_length = 0
for sentence in total_sentences:
    #print(sentence)
    length = len(tokenizer.tokenize(sentence))
    if length > max_length:
        max_length  = length
print("max token length is: ",max_length)
# max token length obtained is 50
# bert tokens are limited to 514 bytes.


max token length is:  297


In [8]:
def encoder_generator(sentences,labels):
    
    sent_index = []
    input_ids = []
    attention_masks =[]

    for index,sent in enumerate(sentences):
        
        sent_index.append(index)
        
        encoded_dict = tokenizer.encode_plus(sent,
                                             add_special_tokens=True,
                                             max_length=max_length,
                                             pad_to_max_length=True,
                                             truncation = True,
                                             return_attention_mask=True,
                                             return_tensors='pt')
        input_ids.append(encoded_dict['input_ids'])

        attention_masks.append(encoded_dict['attention_mask'])

    input_ids = torch.cat(input_ids,dim=0)
    attention_masks = torch.cat(attention_masks,dim=0)
    labels = torch.tensor(labels)
    sent_index = torch.tensor(sent_index)

    return sent_index,input_ids,attention_masks,labels

sent_index,input_ids,attention_masks,encoded_label_tensors = encoder_generator(total_sentences,encoded_labels)
test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors = encoder_generator(test_sentences,encoded_test_labels)
print('Original: ', total_sentences[0])
print('Token IDs:', input_ids[0])
#print(encoded_label_tensors)
#print(encoded_test_label_tensors)

C:\Users\Akshaya Gangurde\anaconda3\lib\site-packages\transformers\tokenization_utils_base.py:2198: FutureWarning: The `pad_to_max_length` argument is deprecated and will be removed in a future version, use `padding=True` or `padding='longest'` to pad to the longest sequence in the batch, or use `padding='max_length'` to pad to a max length. In this case, you can give a specific length with `max_length` (e.g. `max_length=45`) or leave max_length to None to pad to the maximal input size of the model (e.g. 512 for Bert).
  warnings.warn(


Original:  1 आदमीं को मारने पर गोडसे आतंकी हो सके है तो17000 सिखो, 5000 भोपाली, 3000 तमिलों का कत्लेआम करवाने वाला भारत रत्न कैसे हो सके है?
Token IDs: tensor([   101,    122,    852,  15552,  40340,  14018,  11267,  32629,  11549,
         13466,  12213,    867,  69334,  35622,    852,  11845,  90696,  10914,
         13220,    898,  38150,  10569,  21042,  34264,  77802,    898,  12878,
         27841,  13718,    117,  17436,    888,  65430,  88113,    117,  15335,
         37843,  11497,  11081,    865,  11845,  50101,  11554, 111193,  13841,
         16192,  28960,  13466,  62332,  14311,    891,  11845,  22949,    865,
         18438,  35622,  13220,    898,  38150,  10569,    136,    102,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0,      0,      0,      0,      0,
             0,      0,      0,      0,      0, 

In [9]:
from torch.utils.data import TensorDataset,random_split

dataset = TensorDataset(input_ids,attention_masks,encoded_label_tensors)
test_dataset = TensorDataset(test_sent_index,test_input_ids,test_attention_masks,encoded_test_label_tensors)

train_size = int(0.75*len(dataset))

val_size = len(dataset)-train_size

train_dataset,val_dataset = random_split(dataset,[train_size,val_size])

print('train data samples is {}'.format(len(train_dataset)))
print("valid data samples is {}".format(len(val_dataset)))
print("test data samples is {}".format(len(test_dataset)))

train data samples is 6709
valid data samples is 2237
test data samples is 663


In [10]:
from torch.utils.data import DataLoader,RandomSampler,SequentialSampler

bs=8

train_data_loader = DataLoader(train_dataset,
                              sampler=RandomSampler(train_dataset),
                              batch_size=bs)
valid_data_loader = DataLoader(val_dataset,
                              sampler=SequentialSampler(val_dataset),
                              batch_size=bs)
test_data_loader = DataLoader(test_dataset,
                            sampler=SequentialSampler(test_dataset),
                            batch_size=bs)

In [13]:
import torch
torch.cuda.is_available()

False

In [14]:
conda install pytorch torchvision torchaudio cudatoolkit=10.2 -c pytorch

Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.
Solving environment: ...working... failed with initial frozen solve. Retrying with flexible solve.

Note: you may need to restart the kernel to use updated packages.



PackagesNotFoundError: The following packages are not available from current channels:

  - gangurde\anaconda3

Current channels:

  - https://conda.anaconda.org/pytorch/win-64
  - https://conda.anaconda.org/pytorch/noarch
  - https://repo.anaconda.com/pkgs/main/win-64
  - https://repo.anaconda.com/pkgs/main/noarch
  - https://repo.anaconda.com/pkgs/r/win-64
  - https://repo.anaconda.com/pkgs/r/noarch
  - https://repo.anaconda.com/pkgs/msys2/win-64
  - https://repo.anaconda.com/pkgs/msys2/noarch

To search for alternate channels that may provide the conda package you're
looking for, navigate to

    https://anaconda.org

and use the search bar at the top of the page.




In [12]:
from transformers import BertForSequenceClassification, AdamW

model = BertForSequenceClassification.from_pretrained('bert-base-multilingual-cased',
                                                     num_labels=len(le.classes_),
                                                     output_attentions=False,
                                                     output_hidden_states=False,
                                                     )
model.cuda()

Some weights of the model checkpoint at bert-base-multilingual-cased were not used when initializing BertForSequenceClassification: ['cls.predictions.transform.LayerNorm.weight', 'cls.predictions.decoder.weight', 'cls.predictions.transform.dense.bias', 'cls.seq_relationship.weight', 'cls.predictions.bias', 'cls.predictions.transform.dense.weight', 'cls.predictions.transform.LayerNorm.bias', 'cls.seq_relationship.bias']
- This IS expected if you are initializing BertForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing BertForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of BertForSequenceClassification were not initialized from the model ch

AssertionError: Torch not compiled with CUDA enabled